# Transformation Methods — Examples 3.1 and 3.2

This notebook demonstrates sampling using:
- Example 3.1: Exponential via inverse CDF
- Example 3.2: Cauchy via tangent transform

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from mc_lab.transformation_methods import (
    sample_cauchy_via_tangent,
    sample_exponential_via_inverse,
)

## Example 3.1: Exponential via inverse CDF
If U ~ Uniform(0,1), then X = -ln(1-U)/lambda ~ Exponential(rate=lambda).

In [ ]:
# Draw samples
n = 100_000
rate = 2.0
x = sample_exponential_via_inverse(n, rate=rate, random_state=42)

# Plot histogram and overlay theoretical PDF
bins = 80
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].hist(x, bins=bins, density=True, alpha=0.6, color="C0")
xs = np.linspace(0, np.quantile(x, 0.99), 400)
pdf = rate * np.exp(-rate * xs)
ax[0].plot(xs, pdf, "r-", lw=2, label="Exp PDF")
ax[0].set_title("Exponential samples (hist) with PDF overlay")
ax[0].set_xlabel("x")
ax[0].set_ylabel("density")
ax[0].legend()

# Empirical CDF vs theory
xs2 = np.linspace(0, np.quantile(x, 0.999), 400)
ecdf_x = np.sort(x)
ecdf_y = np.arange(1, n + 1) / n
ax[1].plot(ecdf_x, ecdf_y, label="Empirical CDF")
cdf = 1.0 - np.exp(-rate * xs2)
ax[1].plot(xs2, cdf, "r--", label="Theoretical CDF")
ax[1].set_title("Exponential empirical vs theoretical CDF")
ax[1].set_xlabel("x")
ax[1].set_ylabel("F(x)")
ax[1].legend()
plt.tight_layout()
plt.show()

print("Sample mean ~", x.mean(), "; theory =", 1.0 / rate)
print(
    "P(X <= 1/rate) empirical ~", np.mean(x <= 1.0 / rate), "; theory ~", 1 - np.e**-1
)

## Example 3.2: Cauchy via tangent transform
If U ~ Uniform(0,1), then X = tan(π (U - 1/2)) ~ Cauchy(0,1).
More generally X = loc + scale * tan(π (U - 1/2)).

In [ ]:
# Draw samples
n = 200_000
loc, scale = 0.0, 1.5
y = sample_cauchy_via_tangent(n, location=loc, scale=scale, random_state=7)

# Plot histogram (truncated range to visualize heavy tails)
q = 0.99
cut = np.quantile(np.abs(y - loc), q)
mask = np.abs(y - loc) <= cut
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].hist(y[mask], bins=200, density=True, alpha=0.6, color="C1")
xs = np.linspace(loc - cut, loc + cut, 400)
pdf = (1 / np.pi) * (scale / (scale**2 + (xs - loc) ** 2))
ax[0].plot(xs, pdf, "k-", lw=2, label="Cauchy PDF")
ax[0].set_title("Cauchy samples (truncated) with PDF overlay")
ax[0].set_xlabel("x")
ax[0].set_ylabel("density")
ax[0].legend()

# Check the 50% mass within one scale from loc
frac = np.mean(np.abs(y - loc) <= scale)
ax[1].bar(["Empirical"], [frac], color="C2")
ax[1].hlines(0.5, -0.5, 0.5, colors="r", linestyles="--", label="Theory 0.5")
ax[1].set_ylim(0, 1)
ax[1].set_title("Mass within +/- scale of location")
ax[1].legend()
plt.tight_layout()
plt.show()

print("Median (empirical) ~", np.median(y), "; location =", loc)
print("P(|X-loc| <= scale) empirical ~", frac, "; theory = 0.5")